In [ ]:
!apt update
!apt install -y ffmpeg curl
!pip install flask

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.2 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/u

In [ ]:
!mkdir -p templates

In [ ]:
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="en">
<head>
<link rel="shortcut icon" href="https://cdn-icons-png.flaticon.com/512/4221/4221484.png" type="image/png">
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Colab HLS Player</title>
<script src="https://cdn.jsdelivr.net/npm/hls.js@latest"></script>

<style>
/* ---------------- Base Styles ---------------- */
body {
  font-family: "Segoe UI", Arial, sans-serif;
  background: #0d0d0d;
  color: #eaeaea;
  margin: 0;
  padding: 0;
}
p
.container {
  max-width: 1000px;
  margin: auto;
  padding: 30px 20px;
}

/* ---------------- Heading ---------------- */
h1 {
  text-align: center;
  color: #1DB954;
  margin-bottom: 30px;
  font-size: 2rem;
  letter-spacing: 1px;
}

/* ---------------- Form ---------------- */
form {
  display: flex;
  flex-wrap: wrap;
  justify-content: center;
  gap: 12px;
  margin-bottom: 30px;
}

input[type="url"], input[type="text"] {
  flex: 1;
  min-width: 200px;
  padding: 12px;
  border-radius: 8px;
  border: none;
  background: #1e1e1e;
  color: #fff;
  font-size: 0.95rem;
  outline: none;
}

button {
  padding: 12px 22px;
  border: none;
  border-radius: 8px;
  cursor: pointer;
  background: #1DB954;
  color: #fff;
  font-weight: bold;
  font-size: 0.95rem;
  transition: all 0.3s ease;
}
button:hover {
  background: #17a44b;
  transform: translateY(-2px);
  box-shadow: 0 4px 12px rgba(0,0,0,0.3);
}

/* ---------------- Video Grid ---------------- */
.video-list {
  display: grid;
  grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
  gap: 25px;
  margin-top: 20px;
}

.video-card {
  background: #1a1a1a;
  padding: 18px;
  border-radius: 14px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.5);
  transition: all 0.3s ease;
  display: flex;
  flex-direction: column;
  justify-content: space-between;
}
.video-card:hover {
  background: #242424;
  transform: translateY(-3px);
}

/* ---------------- Video Title ---------------- */
.video-card h3 {
  margin: 0;
  font-size: 1.1rem;
  color: #fff;
  margin-bottom: 12px;
  text-overflow: ellipsis;
  overflow: hidden;
  white-space: nowrap;
}



/* ---------------- Play & Delete Button ---------------- */

.button-group {
    display: flex;
    gap: 10px;           /* space between buttons */
    margin-bottom: 10px;
}

.button-group button {
    flex: 1;             /* equal width */
    padding: 10px 0;
    font-weight: bold;
    border-radius: 8px;
    border: none;
    cursor: pointer;
    transition: all 0.3s ease;
    font-size: 0.95rem;
}

/* Play button style */
.play-btn {
    background: #1DB954;
    color: #fff;
}
.play-btn:hover {
    background: #17a44b;
}

/* Delete button style */
.delete-btn {
    background: #ff4d4d;
    color: #fff;
}
.delete-btn:hover {
    background: #e60000;
}

/* ---------------- Progress Bar ---------------- */
.progress-bar {
  width: 0%;
  height: 6px;
  background: #1DB954;
  margin-top: 8px;
  border-radius: 5px;
  transition: width 0.4s ease;
}

/* ---------------- Video Player ---------------- */
video {
  width: 100%;
  margin-top: 12px;
  border-radius: 10px;
  outline: none;
}
</style>
</head>
<body>
<div class="container">
  <h1>🎬 Colab HLS Video Player</h1>
  <form id="downloadForm">
    <input type="url" name="video_url" placeholder="Enter video URL" required>
    <input type="text" name="filename" placeholder="Optional custom name">
    <button type="submit">Download</button>
  </form>

  <div class="video-list" id="videoList">
    {% for video in videos %}
    <div class="video-card" id="{{video}}">
      <h3>{{video}}</h3>
      <div class="button-group">
        <button class="play-btn" onclick="playVideo('{{video}}')">▶ Play</button>
        <button class="delete-btn" onclick="deleteVideo('{{video}}')">🗑 Delete</button>
      </div>
      <div class="progress-bar" id="progress-{{video}}"></div>
      <video id="video-{{video}}" controls></video>
    </div>
    {% endfor %}
  </div>
</div>

<script>
// Submit download form
document.getElementById("downloadForm").onsubmit = async function(e){
    e.preventDefault();
    let url = this.video_url.value;
    let filename = this.filename.value;

    let response = await fetch("/download", {
        method:"POST",
        body: new URLSearchParams({ video_url:url, filename:filename })
    });

    let data = await response.json();
    if(data.status==="started"){
        alert("Download started! Refresh page after some time to see the video.");
    }
}

// Play HLS video
function playVideo(name){
    let video = document.getElementById("video-"+name);
    let videoSrc = `/video/${name}/${name}.m3u8`;

    if(Hls.isSupported()){
        var hls = new Hls();
        hls.loadSource(videoSrc);
        hls.attachMedia(video);
        hls.on(Hls.Events.MANIFEST_PARSED,function(){ video.play(); });
    } else if(video.canPlayType('application/vnd.apple.mpegurl')){
        video.src = videoSrc;
        video.addEventListener('loadedmetadata',function(){ video.play(); });
    }
}

function deleteVideo(name){
    if(!confirm(`Are you sure you want to delete "${name}"?`)){
        return; // user canceled
    }

    fetch(`/delete/${name}`, { method: "POST" })
    .then(res => res.json())
    .then(data => {
        if(data.status === "deleted"){
            alert(`"${name}" deleted successfully!`);
            document.getElementById(name).remove(); // remove card from UI
        } else {
            alert("Error deleting video!");
        }
    })
    .catch(err => alert("Error deleting video!"));
}

</script>
</body>
</html>



Writing templates/index.html


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify, send_from_directory
import subprocess, os, threading, re, hashlib
import shutil

app = Flask(__name__)

# ------------------- Setup ------------------- #
PORT = 9192
HLS_FOLDER = "hls_videos"
os.makedirs(HLS_FOLDER, exist_ok=True)

# ---------------- Helper Functions ---------------- #
def sanitize_filename(url):
    """
    Sanitize the filename: remove unsafe chars, limit length.
    If URL has no meaningful name, fallback to md5 hash.
    """
    name = url.split('/')[-1].split('?')[0]
    name = re.sub(r'[^a-zA-Z0-9_-]', '_', name)
    if not name:
        name = hashlib.md5(url.encode()).hexdigest()
    return name[:50]  # limit to 50 chars


def download_and_convert(video_url, custom_name=None):
    """
    Download the video from the given url give it name if not given already.
    Convert the video to HLS (Https live stream) and delete the original file.
    """
    if custom_name:
        filename = re.sub(r'[^a-zA-Z0-9_-]', '_', custom_name)[:50]
    else:
        filename = sanitize_filename(video_url)

    mp4_path = f"{filename}.mp4"
    folder_path = os.path.join(HLS_FOLDER, filename)
    os.makedirs(folder_path, exist_ok=True)

    # Download video
    subprocess.run(["curl", "-L", video_url, "-o", mp4_path])

    # Convert to HLS
    subprocess.run([              #uses normal copy
        "ffmpeg", "-i", mp4_path,
        "-c:v", "copy",
        "-c:a", "copy",
        "-start_number", "0",
        "-hls_time", "10",
        "-hls_list_size", "0",
        "-f", "hls",
        os.path.join(folder_path, f"{filename}.m3u8")
    ])

#     subprocess.run([
#     "ffmpeg",
#     "-i", mp4_path,
#     "-c:v", "h264_nvenc",    # Use the NVIDIA hardware encoder
#     "-c:a", "aac",
#     "-hls_time", "10",        # Create uniform 6-second segments
#     "-hls_list_size", "0",
#     "-hls_segment_filename", os.path.join(folder_path, f"{filename}_%03d.ts"),
#     os.path.join(folder_path, f"{filename}.m3u8")
# ], check=True)
    os.remove(mp4_path)
    return filename

# ---------------- Flask Routes ---------------- #
@app.route("/")
def index():
    videos = os.listdir(HLS_FOLDER)
    return render_template("index.html", videos=videos)

@app.route("/download", methods=["POST"])
def download():
    video_url = request.form.get("video_url")
    custom_name = request.form.get("filename")  # <-- new field

    if not video_url:
        return jsonify({"status": "error", "message": "No URL provided"}), 400

    thread = threading.Thread(target=download_and_convert, args=(video_url, custom_name))
    thread.start()

    return jsonify({"status": "started"})


@app.route("/video/<video_name>/<path:filename>")
def serve_video(video_name, filename):
    return send_from_directory(os.path.join(HLS_FOLDER, video_name), filename)



@app.route("/delete/<video_name>", methods=["POST"])
def delete_video(video_name):
    folder_path = os.path.join(HLS_FOLDER, video_name)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # delete the folder with .ts + .m3u8
        return jsonify({"status":"deleted"})
    else:
        return jsonify({"status":"error", "message":"Video not found"}), 404


app.run(host="0.0.0.0", port=9192,threaded=True, use_reloader=False)


Writing app.py


In [ ]:
# Start Flask in background + keep logs visible
import subprocess, time

flask_proc = subprocess.Popen(["python", "app.py"])
print("✅ Flask starting in background... Waiting 3s.")
time.sleep(3)

# Now start Serveo tunnel (interactive, will print public link)
!ssh -o StrictHostKeyChecking=no -R 80:localhost:9192 serveo.net


✅ Flask starting in background... Waiting 3s.
ssh: connect to host serveo.net port 22: Connection refused


In [ ]:
!npm install -g localtunnel
!python app.py &
!lt --port 9192

⠙⠹⠸⠼⠴
changed 22 packages in 816ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴ * Serving Flask app 'app'
 * Debug mode: off
Address already in use
Port 9192 is in use by another program. Either identify and stop that program, or start the server with a different port.
your url is: https://empty-hairs-take.loca.lt


In [ ]:
!curl ifconfig.me

35.247.146.169